In [ ]:
# import packages
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import minimize
matplotlib.rcParams['text.usetex'] = True

In [ ]:
# closed form solutions of the incompressible second-order Ogden model, returns engineering stress
def Ogden_uniax(eng_strain, params):
    LA = eng_strain + 1
    m1 = params[0]
    a1 = params[1]
    m2 = params[2]
    a2 = params[3] 
    return 2*m1/a1*(LA**(a1-1)-LA**(-a1/2-1)) + 2*m2/a2*(LA**(a2-1)-LA**(-a2/2-1))

def Ogden_equibiax(eng_strain, params):
    LA = eng_strain + 1
    m1 = params[0]
    a1 = params[1]
    m2 = params[2]
    a2 = params[3] 
    return 2*m1/a1*(LA**(a1-1)-LA**(-2*a1-1)) + 2*m2/a2*(LA**(a2-1)-LA**(-2*a2-1))
    
def Ogden_planar(eng_strain, params):
    LA = eng_strain + 1
    m1 = params[0]
    a1 = params[1]
    m2 = params[2]
    a2 = params[3] 
    return 2*m1/a1*(LA**(a1-1)-LA**(-a1-1)) + 2*m2/a2*(LA**(a2-1)-LA**(-a2-1))

In [ ]:
# defining the error functions 
def Quniax(params):
	eng_stress_sim = Ogden_uniax(eng_strain, params)
	rmse = np.sqrt( 1/len(eng_strain) * sum( (1 - eng_stress_sim[1:]/eng_stress_uniax[1:])**2 ) )
	return rmse

def Qequibiax(params):
	eng_stress_sim = Ogden_equibiax(eng_strain, params)
	rmse = np.sqrt( 1/len(eng_strain) * sum( (1 - eng_stress_sim[1:]/eng_stress_equibiax[1:])**2 ) )
	return rmse

def Qplanar(params):
	eng_stress_sim = Ogden_planar(eng_strain, params)
	rmse = np.sqrt( 1/len(eng_strain) * sum( (1 - eng_stress_sim[1:]/eng_stress_planar[1:])**2 ) )
	return rmse

def Qall(params):
	rmse_uniax = Quniax(params)
	rmse_equibiax = Qequibiax(params)
	rmse_planar = Qplanar(params)
	rmse_all = (rmse_uniax + rmse_equibiax + rmse_planar) / 3
	return rmse_all

In [ ]:
# defining the subrutines for plotting
def comparePlot(stretch, eng_stress_meas, eng_stress_sim, legends, name):
    plt.figure(figsize=(12/2.54, 8/2.54))
    plt.scatter(stretch, eng_stress_meas, s=15, zorder=1, color='red')
    plt.plot(stretch, eng_stress_sim, linewidth=1.75)
    plt.xlabel('$\\mathrm{Stretch, ~ \\lambda (1)}$', labelpad=10, size=12)
    plt.ylabel('$\\mathrm{Engineering ~ stress, ~ P ~ (MPa)}$', labelpad=10, size=12)
    plt.legend(legends, fontsize=12)
    plt.ylim(0,213)
    plt.xlim(1,4)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid('on', linestyle='--', zorder=-2, alpha=0.5)
    plt.savefig(name,bbox_inches='tight',pad_inches=0/25.4)

def Plot(stretch, datalist, legends, name):
    plt.figure(figsize=(12/2.54, 8/2.54))
    for i in datalist:
        plt.plot(stretch, i, linewidth=1.75, linestyle='--')
    for i in datalist:
        plt.scatter(stretch, i, s=15, zorder=1)
    plt.xlabel('$\\mathrm{Stretch, ~ \\lambda (1)}$', labelpad=10, size=12)
    plt.ylabel('$\\mathrm{Engineering ~ stress, ~ P ~ (MPa)}$', labelpad=10, size=12)
    plt.legend(legends, fontsize=12)
    plt.ylim(0,213)
    plt.xlim(1,4)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid('on', linestyle='--', zorder=-2, alpha=0.5)
    plt.savefig(name,bbox_inches='tight',pad_inches=0/25.4)

In [ ]:
# importing the test data
uniax = pd.read_csv('TEST_DATA/HH7AOB_CM_HW02_Uniax.csv',header=None).to_numpy()
equibiax = pd.read_csv('TEST_DATA/HH7AOB_CM_HW02_Equibiax.csv',header=None).to_numpy()
planar = pd.read_csv('TEST_DATA/HH7AOB_CM_HW02_Planar.csv',header=None).to_numpy()

# sorting the values
eng_strain = uniax[:,0]
eng_stress_uniax = uniax[:,1]
eng_stress_equibiax = equibiax[:,1]
eng_stress_planar = planar[:,1]

In [ ]:
## FITTING FOR THE UNIAX TEST DATA ONLY ##
print('\n----------- FITTING FOR THE UNIAX TEST DATA ONLY -----------\n')

# initial guess and optimization of parameters
p0 = np.array([2, 1, 4, 1.2])
bounds = ((0, 100), (-20, 20), (0, 100), (-20, 20))
opt_uniax = minimize(Quniax, p0, method='SLSQP', bounds=bounds, options={'ftol': 1e-30,'disp': True, 'maxiter': 500})
print(opt_uniax)

# obtain the optimized parameters
opt_uniax_params = opt_uniax.x
m1_u = round(opt_uniax_params[0], 5)
a1_u = round(opt_uniax_params[1], 5)
m2_u = round(opt_uniax_params[2], 5)
a2_u = round(opt_uniax_params[3], 5)
print(f'\nThe obtained parameters (UNIAX fit): \nm1 = {m1_u} MPa, \na1 = {a1_u}, \nm2 = {m2_u} MPa, \na2 = {a2_u}')

# the value of the Q parameters
Quniax_u = Quniax([m1_u, a1_u, m2_u, a2_u])
print(f'\nThe value of Quniax for the UNIAX ONLY fit: Quniax = {round(Quniax_u*100, 3)} %.')

Qequibiax_u = Qequibiax([m1_u, a1_u, m2_u, a2_u])
print(f'\nThe value of Qequibiax for the UNIAX ONLY fit: Qequibiax = {round(Qequibiax_u*100, 3)} %.')

Qplanar_u = Qplanar([m1_u, a1_u, m2_u, a2_u])
print(f'\nThe value of Qplanar for the UNIAX ONLY fit: Qplanar = {round(Qplanar_u*100, 3)} %.')

In [ ]:
# comparing the measurements with the fitted model on graphs
stretch = eng_strain + 1

Plot(stretch, [eng_stress_uniax, eng_stress_equibiax, eng_stress_planar], ['Uniaxial test data', 'Equibiaxial test data', 'Planar test data'], 'TASK1_test_data.pdf')
comparePlot(stretch, eng_stress_uniax, Ogden_uniax(eng_strain, opt_uniax_params), ['Uniaxial test data', 'Uniaxial model'], 'TASK4_uniax_fit_uniax.pdf')
comparePlot(stretch, eng_stress_equibiax, Ogden_equibiax(eng_strain, opt_uniax_params), ['Equibiaxial test data', 'Equibiaxial model'], 'TASK4_uniax_fit_equibiax.pdf')
comparePlot(stretch, eng_stress_planar, Ogden_planar(eng_strain, opt_uniax_params), ['Planar test data', 'Planar model'], 'TASK4_uniax_fit_planar.pdf')

In [ ]:
## FITTING FOR ALL TEST DATA ##
print('\n----------- FITTING FOR ALL TEST DATA -----------\n')

# initial guess and optimization of parameters
p0 = np.array([2, 1, 4, -1])
bounds = ((0, 100), (-20, 20), (0, 100), (-20, 20))
opt_all = minimize(Qall, p0, method='SLSQP', bounds=bounds, tol=1e-30, options={'disp': True, 'maxiter': 500})
print(opt_all)

# obtain the optimized parameters
opt_all_params = opt_all.x
m1_a = round(opt_all_params[0], 5)
a1_a = round(opt_all_params[1], 5)
m2_a = round(opt_all_params[2], 5)
a2_a = round(opt_all_params[3], 5)
print(f'\nThe obtained parameters (ALL fit): \nm1 = {m1_a} MPa, \na1 = {a1_a}, \nm2 = {m2_a} MPa, \na2 = {a2_a}')

# the value of the Q parameters
Quniax_a = Quniax([m1_a, a1_a, m2_a, a2_a])
print(f'\nThe value of Quniax for ALL fit: Quniax = {round(Quniax_a*100, 3)} %.')

Qequibiax_a = Qequibiax([m1_a, a1_a, m2_a, a2_a])
print(f'\nThe value of Qequibiax for ALL fit: Qequibiax = {round(Qequibiax_a*100, 3)} %.')

Qplanar_a = Qplanar([m1_a, a1_a, m2_a, a2_a])
print(f'\nThe value of Qplanar for ALL fit: Qplanar = {round(Qplanar_a*100, 3)} %.')

Qall_a = Qall([m1_a, a1_a, m2_a, a2_a])
print(f'\nThe value of Qall for ALL fit: Qall = {round(Qall_a*100, 3)} %.')

In [ ]:
# comparing the measurements with the fitted model on graphs
stretch = eng_strain + 1

comparePlot(stretch, eng_stress_uniax, Ogden_uniax(eng_strain, opt_all_params), ['Uniaxial test data', 'Simultaneous fit'], 'TASK6_all_fit_uniax.pdf')
comparePlot(stretch, eng_stress_equibiax, Ogden_equibiax(eng_strain, opt_all_params), ['Equibiaxial test data', 'Simultaneous fit'], 'TASK6_all_fit_equibiax.pdf')
comparePlot(stretch, eng_stress_planar, Ogden_planar(eng_strain, opt_all_params), ['Planar test data', 'Simultaneous fit'], 'TASK6_all_fit_planar.pdf')

In [ ]:
## TASK 7 ##
print('\n----------- PERFORMING TASK 7 -----------\n')

# declare symbol for parameter "k"
k, X, Y, Z = sp.symbols('k, X, Y, Z')

# declaring displacement field
U = sp.Matrix([ [2-k*Y], [-0.6*X+0.1*Y], [-2] ])

# computing deformation gradient
F_sym = sp.Matrix([ [U[0].diff(X), U[0].diff(Y), U[0].diff(Z)], [U[1].diff(X), U[1].diff(Y), U[1].diff(Z)], [U[2].diff(X), U[2].diff(Y), U[2].diff(Z)] ]) + sp.eye(3)

# calculating parameter "k"
k_val = (sp.solve(sp.det(F_sym)-1, k))[0]
print(f'The value of parameter k is: k = {round(k_val, 3)}.')

In [ ]:
## TASK 8 ##
print('\n----------- PERFORMING TASK 8 -----------\n')

# calculating the left Cauchy-Green deformation tensor
F = F_sym.subs(k,k_val)
b = F*np.transpose(F)

# calculating the principal stretches
eig = b.eigenvects()

lambda1 = sp.sqrt(eig[0][0])
lambda2 = sp.sqrt(eig[1][0])
lambda3 = sp.sqrt(eig[2][0])

n1 = eig[0][2][0]
n2 = eig[1][2][0]
n3 = eig[2][2][0]

print(f'The principal stretches are:\nlambda1 = {lambda1.evalf(4)},\nlambda2 = {lambda2.evalf(4)},\nlambda3 = {lambda3.evalf(4)}.\n')
print(f'The corresponding Eulerian directions:\nn1 = {n1.evalf(4)},\nn2 = {n2.evalf(4)},\nn3 = {n3.evalf(4)}.')

In [ ]:
## TASK 9 ##
print('\n----------- PERFORMING TASK 9 -----------\n')

# strain energy function of incompressible second order Ogden model
la1, la2, la3, m1, m2, a1, a2 = sp.symbols('la1, la2, la3, m1, m2, a1, a2')
W = 2*m1/a1**2*(la1**a1+la2**a1+la3**a1-3) + 2*m2/a2**2*(la1**a2+la2**a2+la3**a2-3)

W_1 = W.diff(la1).subs([(la1, lambda1), (m1, m1_a), (a1, a1_a), (m2, m2_a), (a2, a2_a)])
W_2 = W.diff(la2).subs([(la2, lambda2), (m1, m1_a), (a1, a1_a), (m2, m2_a), (a2, a2_a)])
W_3 = W.diff(la3).subs([(la3, lambda3), (m1, m1_a), (a1, a1_a), (m2, m2_a), (a2, a2_a)])

# deviatoric part of the Cauchy stress tensor 
temp = lambda1*W_1*n1*sp.transpose(n1) + lambda2*W_2*n2*sp.transpose(n2) + lambda3*W_3*n3*sp.transpose(n3)
temp_sph = 1/3*np.trace(temp)*np.eye(3)
s = temp - temp_sph
print(f's =\n{s.evalf(5)}')

# hydrostatic part of the Cauchy stress tensor frm sigma33=0 BC
p = -s[2,2]
print(f'\np =\n{p.evalf(4)}')

# Cauchy stress tensor
sigma = s + np.eye(3)*p
print(f'\nsigma =\n{sigma.evalf(4)}')

# Mises equivalent stress
s = np.array(s, dtype=float)
sigma_mises = np.sqrt(3/2*np.tensordot(s,s))
print(f'\nsigma_mises =\n{round(sigma_mises, 4)}')